In [6]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from datetime import datetime
import numpy as np
import re
import pandas as pd
import os
import json
import pandas as pd
from datetime import datetime
import os
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
import json_repair


In [7]:

llm_chatopenai = AzureChatOpenAI(
    api_key = 'ab40db4c57884de3bf7d8517677ab90e',
    api_version = '2023-03-15-preview',
    azure_endpoint = 'https://lucidmvpopenai.openai.azure.com/',
    temperature = 0,
    deployment_name = 'lucidpg'
    )

# response_schemas = [
#     ResponseSchema(name="TopKeywords", description="List of Top 10 keywords that are there in the Job Description")
# ]

response_schemas = [
    ResponseSchema(name="HardSills", description="List of Hard Skill's keywords that are there in the Job Description"),
    ResponseSchema(name="SoftSkills", description="List of Soft Skill's keywords that are there in the Job Description")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"HardSills": string  // List of Hard Skill's keywords that are there in the Job Description
	"SoftSkills": string  // List of Soft Skill's keywords that are there in the Job Description
}
```


In [8]:
industry = 'Data Science'
Job_Profile = 'Data Scientist'
Job_Description = """
We are currently looking for a new Intermediate Data Scientist, to join our Data Insights team and work closely with Clio’s products and business teams.

You thrive on both analytical challenges and working closely with product development and customer-facing professionals. You will collaborate with everyone from product managers to business leaders and developers, and will guide rapid iterations of hypothesis, prioritization, experimentation / analysis and strategy setting – extract valuable insights, enhance our decision-making processes and contribute to the development of innovative financial products.

 

The team:
You will be working alongside a cross-functional team of data scientists, embedded within Clio’s products and business teams, developing AI and ML solutions to understand Clio’s customers, to bring them cutting-edge AI and GenAI products, and to recommend proactive and efficient ways to serve them better. You will play an integral role, enabling business leaders across Clio make rigorous data-driven decisions. You will help our business grow, help our customers succeed, and continuously improve the way we operate.


 

Who you are:
We aren’t looking for just any traditional Data Scientist to join this team. We’re looking for someone who takes data seriously, thrives in a rapid-growth, high-velocity environment, and lives and breathes our values. We’re looking for an innovator and a thought leader! We’re looking for someone who is:

Passionate about driving growth empirically;

Always looking to innovate with data and explore open-ended questions;

Strategically minded and never shies away from a challenge;

Self-motivated and able to work autonomously and collaboratively;

Agile and responsive, and comfortable with constant change.

You will help lay the foundation for this work by ensuring good data quality, data governance, and analytical practices. And you will also be part of our larger Data & AI team for learning, career development, and company-wide data initiatives.

What you'll work on:
Collaborate with the Clio products teams to refine business problems, develop hypotheses, and provide input that drives growth.

Suggest new questions about our business, product, and customers that lead to impactful insights.

Work with other team members to develop predictive AI and ML solutions and deploy them in production.

Apply rigorous statistical analysis and data mining techniques to evaluate impact of different product features and other business initiatives.

Employ statistical analysis, machine learning, GenAI, LLMs, etc. to unlock new product opportunities.

Support scientific thinking in product and business teams by enabling discussions with data, disseminating best practices, and leading by example.

Effectively communicate complex technical concepts and findings to both technical and non-technical audiences.

What you may have:
3+ years applied experience in data science.

The ability to translate business requirements into data science solutions.

Experience in developing analysis in Python and experience with relevant ML libraries and frameworks (e.g., pandas, PyTorch, scikit-learn)

Strong team player mindset, while able to work under your own initiative and prioritize time and tasks effectively.

Excellent written and verbal communication skills.

Ability to write structured SQL queries for answering questions and manipulating data.

Serious bonus points if you have:
Understanding of SaaS business metrics and growth drivers.

Experience with FinTech concepts.

Experience in analytics working with product and user behavior data, e.g., retention or churn analysis

Experience with building ML/AI pipelines and relevant tools (e.g., Kedro, MLFLow)

Experience with large data sets and user behavior data.

Experience with NLP and LLMs.

A graduate degree in a relevant quantitative discipline (computer science, statistics, mathematics, physics, engineering) 

What you will find here:

Compensation is one of the main components of Clio’s Total Rewards Program. We have developed a series of programs and processes to ensure we are creating fair and competitive pay practices that form the foundation of our human and high-performing culture.
 

Some highlights of our Total Rewards program include:

Competitive, equitable salary with top-tier health benefits, dental, and vision insurance 

Hybrid work environment, with expectation for local Clions (Vancouver, Calgary, Toronto, and Dublin) to be in office min. once per week on our Anchor Day. 

Flexible time off policy, with an encouraged 20 days off per year.

$2000 annual counseling benefit

RRSP matching and RESP contribution 

Clioversary recognition program with special acknowledgement at 3, 5, 7, and 10 years​

The expected salary range* for this role is $106,500 to $144,200 CAD. Please note there are a separate set of salary bands for other regions based on local currency.
*Our salary bands are designed to reflect the range of skills and experience needed for the position and to allow room for growth at Clio. For experienced individuals, we typically hire at or around the midpoint of the band. The top portion of the salary band is reserved for employees who demonstrate sustained high performance and impact at Clio. Those who are new to the role may join below the midpoint and develop their skills over time. The final offer amount for this role will be dependent on geographical region, applicable experience, and skillset of the candidate.

Diversity, Inclusion, Belonging and Equity (DIBE) & Accessibility 

Our team shows up as their authentic selves, and are united by our mission. We are dedicated to diversity, equity and inclusion. We pride ourselves in building and fostering an environment where our teams feel included, valued, and enabled to do the best work of their careers, wherever they choose to log in from. We believe that different perspectives, skills, backgrounds, and experiences result in higher-performing teams and better innovation. We are committed to equal employment and we encourage candidates from all backgrounds to apply.

Clio provides accessibility accommodations during the recruitment process. Should you require any accommodation, please let us know and we will work with you to meet your needs."""

### Top Keywords

In [13]:

rel_base_prompt_data = '''Act as a {industry} industry Hiring manager.For a {Job_Profile} Job Profile, Given is the Job Description {Job_Description}.Extract the List of Top 20 Keywords from the given Job Description that should be in the cadidates resume to get shortlisted for the Job .\n{format_instructions}.\n'''

prompt = PromptTemplate(
    template = rel_base_prompt_data,
    input_variables=["industry" ,"Job_Profile","Job_Description"],
    partial_variables={"format_instructions": format_instructions}
)

_input = prompt.format_prompt(industry=industry,Job_Profile = Job_Profile,Job_Description = Job_Description)
output = llm_chatopenai(_input.to_messages())
print(output)


content='```json\n{\n\t"TopKeywords": [\n\t\t"Data Scientist",\n\t\t"AI",\n\t\t"ML",\n\t\t"Python",\n\t\t"SQL",\n\t\t"FinTech",\n\t\t"Analytics",\n\t\t"Statistical Analysis",\n\t\t"Data Mining",\n\t\t"Product Development",\n\t\t"Customer-Facing",\n\t\t"Data Quality",\n\t\t"Data Governance",\n\t\t"GenAI",\n\t\t"LLMs",\n\t\t"Communication Skills",\n\t\t"SaaS",\n\t\t"User Behavior Data",\n\t\t"NLP",\n\t\t"Quantitative Discipline"\n\t]\n}\n```' response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 1303, 'total_tokens': 1434}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-20611933-6065-452a-98a9-e789bc813ec8-0' usage_metadata={'input_tokens': 1303, 'output_tokens': 131, 'total_tokens': 1434}


### Hard Skills & Soft Skills

In [9]:
rel_base_prompt_data_2 = '''Act as a {industry} industry Hiring manager.For a {Job_Profile} Job Profile, Given is the Job Description {Job_Description}.Extract the list of all hard skills and soft skills keywords from the given Job Description that should be in the cadidates resume to get shortlisted for the Job .\n{format_instructions}.\n'''

prompt = PromptTemplate(
    template = rel_base_prompt_data_2,
    input_variables=["industry" ,"Job_Profile","Job_Description"],
    partial_variables={"format_instructions": format_instructions}
)

_input = prompt.format_prompt(industry=industry,Job_Profile = Job_Profile,Job_Description = Job_Description)
output = llm_chatopenai(_input.to_messages())
print(output)


content='```json\n{\n\t"HardSkills": [\n\t\t"data science",\n\t\t"Python",\n\t\t"ML libraries",\n\t\t"pandas",\n\t\t"PyTorch",\n\t\t"scikit-learn",\n\t\t"SQL",\n\t\t"SaaS business metrics",\n\t\t"FinTech concepts",\n\t\t"analytics",\n\t\t"user behavior data",\n\t\t"retention analysis",\n\t\t"churn analysis",\n\t\t"ML/AI pipelines",\n\t\t"Kedro",\n\t\t"MLFLow",\n\t\t"large data sets",\n\t\t"NLP",\n\t\t"LLMs",\n\t\t"computer science",\n\t\t"statistics",\n\t\t"mathematics",\n\t\t"physics",\n\t\t"engineering"\n\t],\n\t"SoftSkills": [\n\t\t"analytical",\n\t\t"collaborative",\n\t\t"innovative",\n\t\t"self-motivated",\n\t\t"agile",\n\t\t"responsive",\n\t\t"strategic",\n\t\t"team player",\n\t\t"prioritization",\n\t\t"communication"\n\t]\n}\n```' response_metadata={'token_usage': {'completion_tokens': 232, 'prompt_tokens': 1329, 'total_tokens': 1561}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-055f0e

In [10]:
extracted_json = re.search(r'```json\n([\s\S]*?)\n```', output.content)
if extracted_json:
    json_string = extracted_json.group(1)
else:
    json_string = "[]"  # Fallback to an empty JSON array in case the extraction fails

# Now, parse the extracted JSON string
data = json_repair.loads(f"[{json_string}]")
data
# # Create a DataFrame
# df_cleaned = pd.DataFrame({item["TableName"]: item["Columns"] for item in data})
# df_cleaned


[{'HardSkills': ['data science',
   'Python',
   'ML libraries',
   'pandas',
   'PyTorch',
   'scikit-learn',
   'SQL',
   'SaaS business metrics',
   'FinTech concepts',
   'analytics',
   'user behavior data',
   'retention analysis',
   'churn analysis',
   'ML/AI pipelines',
   'Kedro',
   'MLFLow',
   'large data sets',
   'NLP',
   'LLMs',
   'computer science',
   'statistics',
   'mathematics',
   'physics',
   'engineering'],
  'SoftSkills': ['analytical',
   'collaborative',
   'innovative',
   'self-motivated',
   'agile',
   'responsive',
   'strategic',
   'team player',
   'prioritization',
   'communication']}]